In [ ]:
# Import necessary libraries
import requests
import re
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import FrenchStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import numpy as np
from transformers import BertModel, BertTokenizer
import torch
from selenium import webdriver
from bs4 import BeautifulSoup

In [33]:
# Set up Selenium WebDriver (make sure you have the appropriate driver installed, e.g., chromedriver)
driver = webdriver.Chrome()

# URL of the Medium search results page
url = "https://medium.com/search?q=intelligence+artificiel"
driver.get(url)

# Give time for JavaScript to load the content
driver.implicitly_wait(10)

# Parse the rendered HTML content
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# Find all divs containing the article links
article_divs = soup.find_all('div', role='link', attrs={'data-href': True})

# Extract the first 10 article URLs
article_urls = [div['data-href'] for div in article_divs[:10]]

# Debugging: Print article_divs to see if they are being found correctly
print(f"Found {len(article_divs)} article divs")

# Debugging: Print article_urls to check the extracted URLs
print("Extracted article URLs:", article_urls)

# Function to scrape article content
def scrape_article(url):
    article_response = requests.get(url)
    article_soup = BeautifulSoup(article_response.content, 'html.parser')
    
    # Get the title (h1 tag)
    title_tag = article_soup.find('h1')
    title = title_tag.get_text() if title_tag else "No Title Found"
    
    # Extract content from h1-h6, p, ul, and li tags
    content = ""
    for tag in article_soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'ul', 'li']):
        content += tag.get_text() + "\n"
    
    return title, content

# Function to clean article content using regex
def clean_content(content):
    # Define the regex patterns to remove unwanted sections
    start_pattern = re.compile(r'Sign up[\s\S]*?Share\n')
    end_pattern = re.compile(r'Written by[\s\S]*?Teams')
    
    # Remove the unwanted sections
    content = re.sub(start_pattern, '', content)
    content = re.sub(end_pattern, '', content)
    
    return content

# Scrape each article and store the cleaned content
corpus = []
for url in article_urls:
    title, content = scrape_article(url)
    cleaned_content = clean_content(content)
    corpus.append(cleaned_content)
    
    print("="*80 + "\n")
    print(f"Title: {title}\n")
    print(f"Content:\n{cleaned_content}\n")
    print("="*80 + "\n")

# Debugging: Print the final corpus to check the cleaned content
print("Corpus:", corpus)


Found 10 article divs
Extracted article URLs: ['https://medium.com/@kiswendsidajulien/lintelligence-artificiel-et-enjeux-1d9ddbe2553e', 'https://medium.com/@elekis/le-mythe-de-lintelligence-artificiel-45c1c8473057', 'https://medium.com/@peter.keates/lintelligence-artificielle-et-les-modèles-d-affaires-2cd082def3a7', 'https://medium.com/ai-short-articles/intelligence-artificielle-multimodale-une-révolution-dans-la-manière-dont-les-machines-d2e34e874c0a', 'https://medium.com/@peter.keates/comment-choisir-son-premier-projet-dintelligence-artificielle-211ab4acdbb8', 'https://medium.com/@foustry/lintelligence-artificielle-représente-t-elle-un-risque-existentiel-59d317e2a894', 'https://medium.com/ai-short-articles/introduction-à-lintelligence-artificielle-générative-évolution-modèles-et-applications-7be880d37c35', 'https://medium.com/@foustry/par-une-décision-du-24-janvier-2024-publiée-au-joue-du-14-février-2024-la-commission-européenne-a-de32a5df1c9c', 'https://medium.com/@peter.keates/comm


Title: L’intelligence artificielle et les modèles d’affaire

Content:
L’intelligence artificielle (IA) peut transformer et innover un modèle d’affaires de plusieurs manières, allant de l’optimisation des opérations à la création de nouveaux services et produits. Voici quelques façons spécifiques par lesquelles l’IA peut impacter positivement un modèle d’affaires :
1. Personnalisation et Expérience Client
L’IA permet aux entreprises de mieux comprendre leurs clients grâce à l’analyse de données massives. Par exemple :
Systèmes de recommandation : Des plateformes comme Netflix et Amazon utilisent des algorithmes de recommandation pour proposer des contenus ou des produits personnalisés, augmentant ainsi les ventes et la fidélité des clients.Chatbots et assistants virtuels : Ils offrent une assistance client 24/7, améliorant l’expérience utilisateur tout en réduisant les coûts de service.
Systèmes de recommandation : Des plateformes comme Netflix et Amazon utilisent des algorithmes de re


Title: Comment choisir son premier projet d’intelligence artificielle

Content:
Le projet vous permet-il de gagner rapidement ?
Utilisez votre premier projet pilote d’IA pour faire tourner la roue dès que possible.Choisissez des projets initiaux qui peuvent être réalisés rapidement (idéalement dans les 6 à 12 mois) et qui ont de grandes chances de succès.Au lieu de réaliser un seul projet pilote, choisissez-en deux ou trois pour augmenter les chances de créer au moins un succès significatif.
Utilisez votre premier projet pilote d’IA pour faire tourner la roue dès que possible.
Choisissez des projets initiaux qui peuvent être réalisés rapidement (idéalement dans les 6 à 12 mois) et qui ont de grandes chances de succès.
Au lieu de réaliser un seul projet pilote, choisissez-en deux ou trois pour augmenter les chances de créer au moins un succès significatif.
Le projet est-il trop banal ou trop lourd ?
Votre projet pilote n’a pas besoin d’être l’application d’IA la plus précieuse, tant qu


Title: Introduction à l’Intelligence Artificielle Générative : Évolution, Modèles et Applications

Content:
1 — Introduction
--
--





Title: Bureau européen de l’intelligence artificielle

Content:
Par une décision du 24 janvier 2024 publiée au JOUE du 14 février 2024, la Commission européenne a créé le Bureau européen de l’intelligence artificielle.
Cette décision fait suite à la proposition de la Commission du 21 avril 2021 pour un règlement établissant des règles harmonisées sur l’intelligence artificielle (IA), dans le but de favoriser son adoption tout en atténuant les #risques associés à certaines de ses utilisations.
Le Bureau a pour mission de surveiller les avancées des modèles d’IA et leur interaction avec la communauté scientifique. Il a une portée mondiale et joue un rôle clé dans les #enquêtes, les #essais et l’#application de la législation sur l’IA.
Ce Bureau fonctionnera conformément aux processus internes de la Commission européenne, et sa création ne devrait pas af


Title: ChatGPT et l’Art de la Conversation : Une Analyse du Potentiel de l’Intelligence Artificielle Générative

Content:
Section 1 — Présentation
--
--




Corpus: ['Dans nos quotidiens s’il y a quelque chose qui fait les gros titres et fascine le grand public, c’est l’intelligence artificielle qui s’est retrouvée au cœur de la stratégie des grandes entreprises du digital. Et cela grâce à un monde hyper connecté où chaque interaction (des appels téléphoniques aux achats) génère d’importantes quantités de données en plus de celles que génère les objets connectés. La technologie qu’est l’IA entre dans le cadre de gestion de ces masses de données. Tout d’abord de quoi s’agit-il quand on parle d’intelligence artificielle?\nOn peut désigner par Intelligence artificielle « l’ensemble des théories et des techniques mises en œuvre en vue de réaliser des machines capables de simuler l’intelligence » si l’on s’en tient à l’encyclopédie Larousse. Il s’agit d’un ensemble de concepts et technolog

In [ ]:
# Set up Selenium WebDriver (make sure you have the appropriate driver installed, e.g., chromedriver)
driver = webdriver.Chrome()

# URL of the Medium search results page
url = "https://medium.com/search?q=intelligence+artificiel"
driver.get(url)

# Give time for JavaScript to load the content
driver.implicitly_wait(10)

# Parse the rendered HTML content
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# Find all divs containing the article links
article_divs = soup.find_all('div', role='link', attrs={'data-href': True})

# Extract the first 10 article URLs
article_urls = [div['data-href'] for div in article_divs[:10]]

# Debugging: Print article_divs to see if they are being found correctly
print(f"Found {len(article_divs)} article divs")

# Debugging: Print article_urls to check the extracted URLs
print("Extracted article URLs:", article_urls)

# Function to scrape article content
def scrape_article(url):
    article_response = requests.get(url)
    article_soup = BeautifulSoup(article_response.content, 'html.parser')
    
    # Get the title (h1 tag)
    title_tag = article_soup.find('h1')
    title = title_tag.get_text() if title_tag else "No Title Found"
    
    # Extract content from h1-h6, p, ul, and li tags
    content = ""
    for tag in article_soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'ul', 'li']):
        content += tag.get_text() + "\n"
    
    return title, content

# Function to clean article content using regex
def clean_content(content):
    # Define the regex patterns to remove unwanted sections
    start_pattern = re.compile(r'Sign up[\s\S]*?Share\n')
    end_pattern = re.compile(r'Written by[\s\S]*?Teams')
    
    # Remove the unwanted sections
    content = re.sub(start_pattern, '', content)
    content = re.sub(end_pattern, '', content)
    
    return content

# Scrape each article and store the cleaned content in separate files
for i, url in enumerate(article_urls):
    title, content = scrape_article(url)
    cleaned_content = clean_content(content)
    
    # Création du nom du fichier basé sur le numéro de l'article
    filename = f"article_{i+1}.txt"
    
    # Écriture du titre et du contenu propre dans le fichier
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(f"Title: {title}\n\n")
        file.write(f"Content:\n{cleaned_content}\n")
        
    print(f"Article {i+1} saved to {filename}")


In [2]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load the small French model
nlp = spacy.load('fr_core_news_sm')

# Initialize French stemmer
stemmer = FrenchStemmer()

# Function for preprocessing the text
def preprocess_text(text):
    print("Original text:")
    print(f"Title: {title}\n")
    # print(f"Content:\n{text[:100]}...\n")
    print("="*80)
    
    # Tokenization with spacy
    doc = nlp(text)
    tokens = [token.text for token in doc]
    print("Tokens after tokenization:")
    print(tokens)
    print("="*80)

    # Remove stopwords with spacy
    tokens_no_stop = [token for token in tokens if not nlp.vocab[token].is_stop]
    print("Tokens after stopword removal (spaCy):")
    print(tokens_no_stop)
    print("="*80)

    # Stemming with nltk
    stemmed_tokens = [stemmer.stem(token) for token in tokens_no_stop]
    print("Tokens after stemming:")
    print(stemmed_tokens)
    print("="*80)

    # Lemmatization with spacy
    lemmas = [token.lemma_ for token in doc if token.text in tokens_no_stop and token.is_alpha]
    print("Tokens after lemmatization:")
    print(lemmas)
    print("="*80)

    # Further remove stopwords with nltk (optional for double-check)
    stop_words = set(stopwords.words('french'))
    lemmas_no_stop = [lemma for lemma in lemmas if lemma.lower() not in stop_words]
    print("Lemmas after additional stopword removal (NLTK):")
    print(lemmas_no_stop)
    print("="*80)

    return lemmas_no_stop

# Assuming 'corpus' is a list of articles obtained from web scraping


# Preprocess each article in the corpus
preprocessed_corpus = [preprocess_text(article) for article in corpus]

# Debugging: Print the preprocessed corpus
print("Preprocessed corpus:")
print(preprocessed_corpus)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abdoul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abdoul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Original text:
Title: ChatGPT et l’Art de la Conversation : Une Analyse du Potentiel de l’Intelligence Artificielle Générative

Tokens after tokenization:
['Dans', 'nos', 'quotidiens', 's’', 'il', 'y', 'a', 'quelque', 'chose', 'qui', 'fait', 'les', 'gros', 'titres', 'et', 'fascine', 'le', 'grand', 'public', ',', 'c’', 'est', 'l’', 'intelligence', 'artificielle', 'qui', 's’', 'est', 'retrouvée', 'au', 'cœur', 'de', 'la', 'stratégie', 'des', 'grandes', 'entreprises', 'du', 'digital', '.', 'Et', 'cela', 'grâce', 'à', 'un', 'monde', 'hyper', 'connecté', 'où', 'chaque', 'interaction', '(', 'des', 'appels', 'téléphoniques', 'aux', 'achats', ')', 'génère', 'd’', 'importantes', 'quantités', 'de', 'données', 'en', 'plus', 'de', 'celles', 'que', 'génère', 'les', 'objets', 'connectés', '.', 'La', 'technologie', 'qu’', 'est', 'l’', 'IA', 'entre', 'dans', 'le', 'cadre', 'de', 'gestion', 'de', 'ces', 'masses', 'de', 'données', '.', 'Tout', 'd’', 'abord', 'de', 'quoi', 's’', 'agit', '-il', 'quand', '

Tokens after tokenization:
['(', 'english', 'version', 'here', ')', '\n', 'Il', 'marrant', 'd’', 'entendre', 'ici', 'et', 'là', 'de', 'grands', 'penseurs', 'essayer', 'de', 'comprendre', 'le', 'Saint-Graal', 'de', 'toute', 'réussite', 'sociale', 'au', '21e', 'siècle', ':', 'les', 'algorithmes', 'de', 'publications', 'tels', 'que', 'YouTube', ',', 'Facebook', 'ou', 'Twitter', 'pour', 'ne', 'citer', 'que', 'les', 'plus', 'connus', '.', '\n', 'Il', 'existe', 'un', 'mythe', 'qu’', 'une', 'intelligence', 'artificielle', 'serait', 'capable', 'de', 'déceler', 'ce', 'qui', 'est', 'bon', 'pour', 'vous', 'après', 'avoir', 'violé', 'toutes', 'vos', 'données', '.', 'Si', 'seulement', 'ces', 'youtubeurs', 'en', 'herbe', 'savaient', 'la', 'vérité', 'qui', 'n’', 'est', 'pas', 'compliquée', 'en', 'soi', ',', 'mais', 'qui', ',', 'dans', 'le', 'marasme', 'des', 'informations', 'non', 'divulguées', 'par', 'les', 'gérants', ',', 'fait', 'fantasmer', 'de', 'complexe', 'calcul', '.', '\n', 'Intelligence', '

Tokens after tokenization:
['L’', 'intelligence', 'artificielle', '(', 'IA', ')', 'peut', 'transformer', 'et', 'innover', 'un', 'modèle', 'd’', 'affaires', 'de', 'plusieurs', 'manières', ',', 'allant', 'de', 'l’', 'optimisation', 'des', 'opérations', 'à', 'la', 'création', 'de', 'nouveaux', 'services', 'et', 'produits', '.', 'Voici', 'quelques', 'façons', 'spécifiques', 'par', 'lesquelles', 'l’', 'IA', 'peut', 'impacter', 'positivement', 'un', 'modèle', 'd’', 'affaires', ':', '\n', '1', '.', 'Personnalisation', 'et', 'Expérience', 'Client', '\n', 'L’', 'IA', 'permet', 'aux', 'entreprises', 'de', 'mieux', 'comprendre', 'leurs', 'clients', 'grâce', 'à', 'l’', 'analyse', 'de', 'données', 'massives', '.', 'Par', 'exemple', ':', '\n', 'Systèmes', 'de', 'recommandation', ':', 'Des', 'plateformes', 'comme', 'Netflix', 'et', 'Amazon', 'utilisent', 'des', 'algorithmes', 'de', 'recommandation', 'pour', 'proposer', 'des', 'contenus', 'ou', 'des', 'produits', 'personnalisés', ',', 'augmentant', 'a

Tokens after tokenization:
['Le', 'projet', 'vous', 'permet', '-il', 'de', 'gagner', 'rapidement', '?', '\n', 'Utilisez', 'votre', 'premier', 'projet', 'pilote', 'd’', 'IA', 'pour', 'faire', 'tourner', 'la', 'roue', 'dès', 'que', 'possible', '.', 'Choisissez', 'des', 'projets', 'initiaux', 'qui', 'peuvent', 'être', 'réalisés', 'rapidement', '(', 'idéalement', 'dans', 'les', '6', 'à', '12', 'mois', ')', 'et', 'qui', 'ont', 'de', 'grandes', 'chances', 'de', 'succès', '.', 'Au', 'lieu', 'de', 'réaliser', 'un', 'seul', 'projet', 'pilote', ',', 'choisissez', '-en', 'deux', 'ou', 'trois', 'pour', 'augmenter', 'les', 'chances', 'de', 'créer', 'au', 'moins', 'un', 'succès', 'significatif', '.', '\n', 'Utilisez', 'votre', 'premier', 'projet', 'pilote', 'd’', 'IA', 'pour', 'faire', 'tourner', 'la', 'roue', 'dès', 'que', 'possible', '.', '\n', 'Choisissez', 'des', 'projets', 'initiaux', 'qui', 'peuvent', 'être', 'réalisés', 'rapidement', '(', 'idéalement', 'dans', 'les', '6', 'à', '12', 'mois', '

Tokens after tokenization:
['Questions', 'et', 'réponses', 'avec', 'des', 'experts', 'de', 'la', 'RAND', 'Corporation', '(', '*', ')', '\n', 'Quels', 'sont', 'les', 'risques', 'potentiels', 'associés', 'à', 'l’', 'intelligence', 'artificielle', '?', 'Certains', 'de', 'ces', 'risques', 'pourraient', '-ils', 'être', 'catastrophiques', ',', 'voire', 'existentiels', '?', '\n', 'Lors', 'd’', 'un', 'événement', 'organisé', 'le', '11', 'mars', '2024', 'par', 'la', 'RAND', 'Corporation', ',', 'un', 'panel', 'de', 'cinq', 'experts', 'a', 'tenté', 'de', 'répondre', 'à', 'ces', 'questions', '.', '\n', '-', 'Benjamin', 'Boudreaux', '(', 'chercheur', 'en', 'politique', 'spécialisé', 'dans', 'l’', 'éthique', ',', 'la', 'technologie', 'émergente', 'et', 'la', 'sécurité', ')', 'souligne', 'que', 'l’', 'IA', 'pourrait', 'représenter', 'un', 'risque', 'significatif', 'pour', 'notre', 'qualité', 'de', 'vie', 'et', 'les', 'institutions', 'nécessaires', 'à', 'notre', 'prospérité', ',', 'similaire', 'à', 'u

Tokens after tokenization:
['Par', 'une', 'décision', 'du', '24', 'janvier', '2024', 'publiée', 'au', 'JOUE', 'du', '14', 'février', '2024', ',', 'la', 'Commission', 'européenne', 'a', 'créé', 'le', 'Bureau', 'européen', 'de', 'l’', 'intelligence', 'artificielle', '.', '\n', 'Cette', 'décision', 'fait', 'suite', 'à', 'la', 'proposition', 'de', 'la', 'Commission', 'du', '21', 'avril', '2021', 'pour', 'un', 'règlement', 'établissant', 'des', 'règles', 'harmonisées', 'sur', 'l’', 'intelligence', 'artificielle', '(', 'IA', ')', ',', 'dans', 'le', 'but', 'de', 'favoriser', 'son', 'adoption', 'tout', 'en', 'atténuant', 'les', '#', 'risques', 'associés', 'à', 'certaines', 'de', 'ses', 'utilisations', '.', '\n', 'Le', 'Bureau', 'a', 'pour', 'mission', 'de', 'surveiller', 'les', 'avancées', 'des', 'modèles', 'd’', 'IA', 'et', 'leur', 'interaction', 'avec', 'la', 'communauté', 'scientifique', '.', 'Il', 'a', 'une', 'portée', 'mondiale', 'et', 'joue', 'un', 'rôle', 'clé', 'dans', 'les', '#', 'enq

Tokens after tokenization:
['L’', 'intelligence', 'artificielle', '(', 'IA', ')', 'est', 'une', 'avancée', 'technologique', 'révolutionnaire', 'qui', 'offre', 'un', 'immense', 'potentiel', 'pour', 'la', 'société', '.', 'L’', 'IA', 'peut', 'améliorer', 'de', 'nombreux', 'aspects', 'de', 'notre', 'vie', ',', 'de', 'la', 'santé', 'à', 'l’', 'éducation', ',', 'en', 'passant', 'par', 'les', 'transports', 'et', 'la', 'gestion', 'des', 'ressources', '.', '\n', 'Comprendre', 'les', 'avantages', 'potentiels', 'de', 'l’', 'IA', '\n', 'L’', 'intelligence', 'artificielle', '(', 'IA', ')', 'est', 'une', 'avancée', 'technologique', 'révolutionnaire', 'qui', 'offre', 'un', 'immense', 'potentiel', 'pour', 'la', 'société', '.', 'L’', 'IA', 'peut', 'améliorer', 'de', 'nombreux', 'aspects', 'de', 'notre', 'vie', ',', 'de', 'la', 'santé', 'à', 'l’', 'éducation', ',', 'en', 'passant', 'par', 'les', 'transports', 'et', 'la', 'gestion', 'des', 'ressources', '.', 'Grâce', 'à', 'l’', 'IA', ',', 'nous', 'pouvon

Tokens after tokenization:
['Section', '1', '—', 'Présentation', '\n', '--', '\n', '--', '\n\n']
Tokens after stopword removal (spaCy):
['Section', '1', '—', 'Présentation', '\n', '--', '\n', '--', '\n\n']
Tokens after stemming:
['section', '1', '—', 'présent', '\n', '--', '\n', '--', '\n\n']
Tokens after lemmatization:
['section', 'présentation']
Lemmas after additional stopword removal (NLTK):
['section', 'présentation']
Preprocessed corpus:
[['quotidien', 'chose', 'gros', 'titre', 'fascine', 'grand', 'public', 'intelligence', 'artificiel', 'retrouver', 'cœur', 'stratégie', 'grand', 'entreprise', 'digital', 'grâce', 'monde', 'hyper', 'connecter', 'interaction', 'appel', 'téléphonique', 'achat', 'générer', 'important', 'quantité', 'donnée', 'générer', 'objet', 'connecter', 'technologie', 'IA', 'cadre', 'gestion', 'masse', 'donnée', 'agir', 'intelligence', 'artificiel', 'désigner', 'intelligence', 'artificiel', 'ensemble', 'théorie', 'technique', 'mettre', 'œuvre', 'vue', 'réaliser', '

In [5]:
# Convert preprocessed corpus back to strings
preprocessed_corpus_str = [" ".join(article) for article in preprocessed_corpus]

# Bag of Words
bow_vectorizer = CountVectorizer()
bow_vectors = bow_vectorizer.fit_transform(preprocessed_corpus_str)

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(preprocessed_corpus_str)

print("BoW Vectors:")
print(bow_vectors.toarray())
print("TF-IDF Vectors:")
print(tfidf_vectors.toarray())


BoW Vectors:
[[0 0 0 ... 2 0 1]
 [0 0 0 ... 1 1 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 3 ... 4 0 0]
 [0 0 0 ... 0 0 0]]
TF-IDF Vectors:
[[0.         0.         0.         ... 0.05588943 0.         0.04705917]
 [0.         0.         0.         ... 0.02636776 0.04440356 0.        ]
 [0.         0.01678786 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.0090327  0.03643541 ... 0.02884814 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [8]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=preprocessed_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Function to get the average Word2Vec vector for a document
def get_avg_word2vec_vector(text):
    vectors = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Get average Word2Vec vectors for each document
word2vec_vectors_avg = [get_avg_word2vec_vector(text) for text in preprocessed_corpus]

print("Word2Vec Vectors:")
for vector in word2vec_vectors_avg:
    print(vector)


Word2Vec Vectors:
[-1.0026791e-03  4.9185538e-04  8.3415146e-04  1.7692657e-04
  4.6146047e-04 -1.2431744e-03  2.2408296e-04  1.3808018e-03
 -8.1558799e-04 -2.5735630e-04 -1.0608039e-03 -1.5334758e-03
 -9.6107447e-05  6.6333479e-04  3.7137381e-04 -9.7129826e-04
  2.8811695e-04 -4.5461769e-04 -2.3819346e-04 -1.2101303e-03
  6.6768320e-04  3.4987647e-04  3.7985848e-04 -5.4171833e-04
  4.2858507e-04  5.9179007e-04 -9.2844450e-04 -4.3242137e-04
 -5.4086425e-04  1.4841928e-04  9.5747155e-04  9.3551941e-04
  4.5118108e-04 -9.1922510e-04 -7.6521590e-04  1.0710231e-03
 -2.4847648e-04 -5.5981352e-04 -8.6141605e-04 -1.1532496e-03
  1.6772902e-06 -8.8626746e-04 -2.3294431e-04 -1.0490431e-03
  9.0102840e-04  8.9232657e-05 -1.0974056e-03  1.1079241e-04
  3.3699640e-04  1.1290317e-06  8.5612765e-04 -1.4292686e-03
 -1.5045983e-04  3.6516917e-04 -9.7252370e-04 -8.3300583e-06
  2.2191048e-04 -4.5486997e-04 -6.1566656e-04  3.3068241e-04
  1.3583113e-04  2.6910959e-04 -6.7739544e-04  5.5017631e-04
 -8.83

In [9]:
# Load pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

bert_vectors = [get_bert_embedding(" ".join(article)) for article in preprocessed_corpus]

print("BERT Vectors:")
for vector in bert_vectors:
    print(vector.numpy())


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT Vectors:
[-5.54125071e-01  1.79481089e-01  3.38814199e-01 -1.22789249e-01
  6.90555692e-01 -3.65950800e-02 -7.19938874e-02  4.26406920e-01
 -1.66829199e-01 -4.72817600e-01 -5.72898239e-02 -1.76992550e-01
 -4.20895070e-02  1.98994815e-01 -2.04022810e-01  6.33264244e-01
 -1.72021054e-02 -1.29148245e-01 -9.27419811e-02  2.38542974e-01
  2.43454382e-01  1.86492503e-02 -1.59119934e-01  5.14368415e-01
  3.18644643e-01 -6.57419413e-02 -1.12231210e-01 -1.15354404e-01
 -5.74717000e-02  1.58673763e-01  1.52379245e-01  1.19111329e-01
 -1.30600363e-01 -3.78712654e-01  2.48389523e-02 -1.98244452e-02
 -4.30298820e-02 -1.07032709e-01 -4.88204062e-02  3.62665385e-01
 -1.73754334e-01 -5.46988010e-01 -8.83098319e-02  1.31708145e-01
 -3.36189985e-01 -3.17663491e-01 -7.68557712e-02  1.20463431e-01
  1.89939439e-01  1.17499553e-01 -7.09278107e-01  3.74427199e-01
  1.31304830e-01 -5.93235046e-02  3.44411612e-01  3.68046045e-01
 -1.58928201e-01 -4.91637856e-01 -2.69689113e-01 -3.06163967e-01
  2.0453631

In [12]:
# Convert preprocessed corpus back to strings
preprocessed_corpus_str = [" ".join(article) for article in preprocessed_corpus]

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(preprocessed_corpus_str)

# Get feature names (words)
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# Get TF-IDF scores for each document
tfidf_scores = tfidf_vectors.toarray()

# Extract top N keywords for each document
top_n = 10
keywords = []
for doc_scores in tfidf_scores:
    top_indices = doc_scores.argsort()[-top_n:]
    doc_keywords = feature_names[top_indices]
    keywords.extend(doc_keywords)

# Remove duplicates and get the most frequent keywords
keywords = list(set(keywords))

print("Extracted Keywords:")
print(keywords)

Extracted Keywords:
['bénéficier', 'grand', 'client', 'peur', 'politique', 'potentiel', 'changement', 'projet', 'décision', 'entièrement', 'intelligence', 'fichier', 'agriculture', 'analyser', 'esprit', 'corporation', 'système', 'bureau', 'émergent', 'the', 'suite', 'section', 'envisager', 'produit', 'ia', 'innovation', 'demande', 'réaliser', 'donnée', 'utilisateur', 'commission', 'technologie', 'entrainée', 'era', 'américain', 'rand', 'œuvre', 'and', 'risque', 'entraîner', 'strategy', 'entreprise', 'vouloir', 'adoption', 'algorithme', 'colonne', 'organe', 'reconnaitre', 'succès', 'erreur', 'garantir', 'entreprendre', 'envisagez', 'affaire', 'présentation', 'essentiel', 'image', 'capable', 'joue', 'introduction', 'règlement', 'entendre', 'société', 'ensemble', 'pilote', 'responsabilité', 'jour', 'investissement', 'service', 'responsable', 'agir', 'artificiel', 'afrique', 'essai', 'emploi', 'européen', 'utilisation', 'business', 'modèle', 'ensembl', 'environnement']


In [16]:
preprocessed_corpus_str = [" ".join(article) for article in preprocessed_corpus]


In [17]:
# Bag of Words
bow_vectorizer = CountVectorizer()
bow_vectors = bow_vectorizer.fit_transform(preprocessed_corpus_str)

print("BoW Vectors:")
print(bow_vectors.toarray())


BoW Vectors:
[[0 0 0 ... 2 0 1]
 [0 0 0 ... 1 1 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 3 ... 4 0 0]
 [0 0 0 ... 0 0 0]]


In [18]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(preprocessed_corpus_str)

print("TF-IDF Vectors:")
print(tfidf_vectors.toarray())


TF-IDF Vectors:
[[0.         0.         0.         ... 0.05588943 0.         0.04705917]
 [0.         0.         0.         ... 0.02636776 0.04440356 0.        ]
 [0.         0.01678786 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.0090327  0.03643541 ... 0.02884814 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [19]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=preprocessed_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Function to get the average Word2Vec vector for a document
def get_avg_word2vec_vector(text):
    vectors = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Get average Word2Vec vectors for each document
word2vec_vectors_avg = [get_avg_word2vec_vector(text) for text in preprocessed_corpus]

print("Word2Vec Vectors:")
for vector in word2vec_vectors_avg:
    print(vector)


Word2Vec Vectors:
[-1.0026791e-03  4.9185538e-04  8.3415146e-04  1.7692657e-04
  4.6146047e-04 -1.2431744e-03  2.2408296e-04  1.3808018e-03
 -8.1558799e-04 -2.5735630e-04 -1.0608039e-03 -1.5334758e-03
 -9.6107447e-05  6.6333479e-04  3.7137381e-04 -9.7129826e-04
  2.8811695e-04 -4.5461769e-04 -2.3819346e-04 -1.2101303e-03
  6.6768320e-04  3.4987647e-04  3.7985848e-04 -5.4171833e-04
  4.2858507e-04  5.9179007e-04 -9.2844450e-04 -4.3242137e-04
 -5.4086425e-04  1.4841928e-04  9.5747155e-04  9.3551941e-04
  4.5118108e-04 -9.1922510e-04 -7.6521590e-04  1.0710231e-03
 -2.4847648e-04 -5.5981352e-04 -8.6141605e-04 -1.1532496e-03
  1.6772902e-06 -8.8626746e-04 -2.3294431e-04 -1.0490431e-03
  9.0102840e-04  8.9232657e-05 -1.0974056e-03  1.1079241e-04
  3.3699640e-04  1.1290317e-06  8.5612765e-04 -1.4292686e-03
 -1.5045983e-04  3.6516917e-04 -9.7252370e-04 -8.3300583e-06
  2.2191048e-04 -4.5486997e-04 -6.1566656e-04  3.3068241e-04
  1.3583113e-04  2.6910959e-04 -6.7739544e-04  5.5017631e-04
 -8.83

In [20]:
# Load pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

bert_vectors = [get_bert_embedding(" ".join(article)) for article in preprocessed_corpus]

print("BERT Vectors:")
for vector in bert_vectors:
    print(vector.numpy())


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT Vectors:
[-5.54125071e-01  1.79481089e-01  3.38814199e-01 -1.22789249e-01
  6.90555692e-01 -3.65950800e-02 -7.19938874e-02  4.26406920e-01
 -1.66829199e-01 -4.72817600e-01 -5.72898239e-02 -1.76992550e-01
 -4.20895070e-02  1.98994815e-01 -2.04022810e-01  6.33264244e-01
 -1.72021054e-02 -1.29148245e-01 -9.27419811e-02  2.38542974e-01
  2.43454382e-01  1.86492503e-02 -1.59119934e-01  5.14368415e-01
  3.18644643e-01 -6.57419413e-02 -1.12231210e-01 -1.15354404e-01
 -5.74717000e-02  1.58673763e-01  1.52379245e-01  1.19111329e-01
 -1.30600363e-01 -3.78712654e-01  2.48389523e-02 -1.98244452e-02
 -4.30298820e-02 -1.07032709e-01 -4.88204062e-02  3.62665385e-01
 -1.73754334e-01 -5.46988010e-01 -8.83098319e-02  1.31708145e-01
 -3.36189985e-01 -3.17663491e-01 -7.68557712e-02  1.20463431e-01
  1.89939439e-01  1.17499553e-01 -7.09278107e-01  3.74427199e-01
  1.31304830e-01 -5.93235046e-02  3.44411612e-01  3.68046045e-01
 -1.58928201e-01 -4.91637856e-01 -2.69689113e-01 -3.06163967e-01
  2.0453631

In [21]:
# Convert preprocessed corpus back to strings
preprocessed_corpus_str = [" ".join(article) for article in preprocessed_corpus]

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(preprocessed_corpus_str)

# Get feature names (words)
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

# Get TF-IDF scores for each document
tfidf_scores = tfidf_vectors.toarray()

# Extract top N keywords for each document
top_n = 10
keywords = []
for doc_scores in tfidf_scores:
    top_indices = doc_scores.argsort()[-top_n:]
    doc_keywords = feature_names[top_indices]
    keywords.extend(doc_keywords)

# Remove duplicates and get the most frequent keywords
keywords = list(set(keywords))

print("Extracted Keywords:")
print(keywords)


Extracted Keywords:
['bénéficier', 'grand', 'client', 'peur', 'politique', 'potentiel', 'changement', 'projet', 'décision', 'entièrement', 'intelligence', 'fichier', 'agriculture', 'analyser', 'esprit', 'corporation', 'système', 'bureau', 'émergent', 'the', 'suite', 'section', 'envisager', 'produit', 'ia', 'innovation', 'demande', 'réaliser', 'donnée', 'utilisateur', 'commission', 'technologie', 'entrainée', 'era', 'américain', 'rand', 'œuvre', 'and', 'risque', 'entraîner', 'strategy', 'entreprise', 'vouloir', 'adoption', 'algorithme', 'colonne', 'organe', 'reconnaitre', 'succès', 'erreur', 'garantir', 'entreprendre', 'envisagez', 'affaire', 'présentation', 'essentiel', 'image', 'capable', 'joue', 'introduction', 'règlement', 'entendre', 'société', 'ensemble', 'pilote', 'responsabilité', 'jour', 'investissement', 'service', 'responsable', 'agir', 'artificiel', 'afrique', 'essai', 'emploi', 'européen', 'utilisation', 'business', 'modèle', 'ensembl', 'environnement']


In [24]:
# Compute cosine similarity between articles and keyword vector
keyword_vector = tfidf_vectorizer.transform([" ".join(keywords)]).toarray()

similarities = cosine_similarity(tfidf_vectors, keyword_vector)

print("Similarities TF-IDF:")
print(similarities)


Similarities TF-IDF:
[[0.21390924]
 [0.26108602]
 [0.29330631]
 [0.        ]
 [0.23600813]
 [0.22087467]
 [0.12332199]
 [0.31065439]
 [0.27725059]
 [0.17440363]]


In [23]:
# Calcul de la similarité cosinus pour BoW
bow_similarities = cosine_similarity(bow_vectors, bow_vectorizer.transform([" ".join(keywords)]))

print("Similarités BoW:")
print(bow_similarities)


Similarités BoW:
[[0.27816919]
 [0.29499826]
 [0.34359852]
 [0.        ]
 [0.28756599]
 [0.26730898]
 [0.11111111]
 [0.34480641]
 [0.29258992]
 [0.15713484]]


In [25]:
# Fonction pour obtenir le vecteur Word2Vec moyen pour une liste de mots-clés
def get_avg_word2vec_vector_from_keywords(keywords):
    vectors = [word2vec_model.wv[word] for word in keywords if word in word2vec_model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Calculer le vecteur moyen pour les mots-clés
keyword_vector_word2vec = get_avg_word2vec_vector_from_keywords(keywords).reshape(1, -1)

# Calcul de la similarité cosinus pour Word2Vec
word2vec_similarities = cosine_similarity(word2vec_vectors_avg, keyword_vector_word2vec)

print("Similarités Word2Vec:")
print(word2vec_similarities)


Similarités Word2Vec:
[[0.70527715]
 [0.76655632]
 [0.78832575]
 [0.        ]
 [0.65749926]
 [0.76724626]
 [0.00703736]
 [0.70100753]
 [0.79726007]
 [0.12626904]]


In [28]:
# Convertir la liste de mots-clés en une chaîne de caractères
keywords_str = " ".join(keywords)

# Obtenir l'embedding BERT pour les mots-clés
keyword_vector_bert = get_bert_embedding(keywords_str).unsqueeze(0)

# Convertir les embeddings BERT des articles en une matrice
bert_matrix = torch.stack(bert_vectors)

# Calcul de la similarité cosinus pour BERT
bert_similarities = cosine_similarity(bert_matrix, keyword_vector_bert)

print("Similarités BERT:")
print(bert_similarities)



Similarités BERT:
[[0.895796  ]
 [0.87818635]
 [0.80738616]
 [0.12829559]
 [0.8314463 ]
 [0.8612825 ]
 [0.2007026 ]
 [0.9484725 ]
 [0.8410038 ]
 [0.28422943]]


In [34]:
# Résultats de similarité
tfidf_similarities = np.array([[0.21390924], [0.26108602], [0.29330631], [0.0], [0.23600813], [0.22087467], [0.12332199], [0.31065439], [0.27725059], [0.17440363]])
bow_similarities = np.array([[0.27816919], [0.29499826], [0.34359852], [0.0], [0.28756599], [0.26730898], [0.11111111], [0.34480641], [0.29258992], [0.15713484]])
word2vec_similarities = np.array([[0.70527715], [0.76655632], [0.78832575], [0.0], [0.65749926], [0.76724626], [0.00703736], [0.70100753], [0.79726007], [0.12626904]])
bert_similarities = np.array([[0.70527715], [0.76655632], [0.78832575], [0.0], [0.65749926], [0.76724626], [0.00703736], [0.70100753], [0.79726007], [0.12626904]])

# Fonction pour trier les articles par similarité
def get_top_n_indices(similarities, n=10):
    return similarities.squeeze().argsort()[::-1][:n]

# Obtenir les indices des 10 premiers articles pour chaque technique
top_n_tfidf = get_top_n_indices(tfidf_similarities)
top_n_bow = get_top_n_indices(bow_similarities)
top_n_word2vec = get_top_n_indices(word2vec_similarities)
top_n_bert = get_top_n_indices(bert_similarities)

# Afficher les résultats
print("Top 10 articles par similarité TF-IDF:", top_n_tfidf)
print("Top 10 articles par similarité BoW:", top_n_bow)
print("Top 10 articles par similarité Word2Vec:", top_n_word2vec)
print("Top 10 articles par similarité BERT:", top_n_bert)


Top 10 articles par similarité TF-IDF: [7 2 8 1 4 5 0 9 6 3]
Top 10 articles par similarité BoW: [7 2 1 8 4 0 5 9 6 3]
Top 10 articles par similarité Word2Vec: [8 2 5 1 0 7 4 9 6 3]
Top 10 articles par similarité BERT: [8 2 5 1 0 7 4 9 6 3]


In [35]:
# Résultats de similarité
tfidf_similarities = np.array([[0.21390924], [0.26108602], [0.29330631], [0.0], [0.23600813], [0.22087467], [0.12332199], [0.31065439], [0.27725059], [0.17440363]])
bow_similarities = np.array([[0.27816919], [0.29499826], [0.34359852], [0.0], [0.28756599], [0.26730898], [0.11111111], [0.34480641], [0.29258992], [0.15713484]])
word2vec_similarities = np.array([[0.70527715], [0.76655632], [0.78832575], [0.0], [0.65749926], [0.76724626], [0.00703736], [0.70100753], [0.79726007], [0.12626904]])
bert_similarities = np.array([[0.70527715], [0.76655632], [0.78832575], [0.0], [0.65749926], [0.76724626], [0.00703736], [0.70100753], [0.79726007], [0.12626904]])

# Calculer les moyennes des similarités pour chaque technique
mean_tfidf_similarity = np.mean(tfidf_similarities)
mean_bow_similarity = np.mean(bow_similarities)
mean_word2vec_similarity = np.mean(word2vec_similarities)
mean_bert_similarity = np.mean(bert_similarities)

# Afficher les résultats
print("Moyenne des similarités TF-IDF:", mean_tfidf_similarity)
print("Moyenne des similarités BoW:", mean_bow_similarity)
print("Moyenne des similarités Word2Vec:", mean_word2vec_similarity)
print("Moyenne des similarités BERT:", mean_bert_similarity)


Moyenne des similarités TF-IDF: 0.21108149699999998
Moyenne des similarités BoW: 0.23772832200000002
Moyenne des similarités Word2Vec: 0.5316478740000001
Moyenne des similarités BERT: 0.5316478740000001
